In [324]:
# "Additional Excision (AE)" - A (100%)

# Inform GPT of what qualifies as an AE and what doesn't in system message, then ask if an AE is required and pass in A notes

import os
import pandas as pd
import requests
import base64

#using azure
API_KEY = "2DWhd5NlWo6U8lpUBSWgyCuCL1bgNUAlQyv4sF2Oq6TsNxszq97NJQQJ99AKACHrzpqXJ3w3AAABACOGzNKu"
ENDPOINT = "https://rg-01-hippa-standalone-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-15-preview"
def call_gpt(clinical_text):
    """Improved function for AE determination"""
    if not clinical_text.strip():
        return "No"  # Default to No if no clinical text
    
    headers = {
        "Content-Type": "application/json",
        "api-key": API_KEY,
    }
    system_context = """
You are a dermatology specialist analyzing ONLY the Addendum section of clinical notes to determine whether Additional Excision (AE) is being planned, recommended, or discussed.

Return "Yes" if ANY of the following appear in the addendum:
- Additional excision is explicitly stated or implied: e.g., “additional excision recommended,” “to return for wider excision,” “needs AE,” “to schedule further excision,” “coordinating AE,” or “awaiting final excision.”
- Pathology results or margin descriptions clearly justify AE, such as:
   - “positive margin” or “positive at deep margin”
   - “tumor extends to base,” “not clear at base”
   - “residual tumor,” “tumor not fully excised”
   - “infiltrative features,” “aggressive subtype”
- The provider communicates to the patient or clinical team that more surgery is needed or likely.
- The plan includes coordination with surgery or multidisciplinary follow-up specifically related to margin or tumor concerns.

Return "No" if:
- The addendum indicates margins are clear, negative, or completely excised with no mention of future excision.
- The follow-up is routine, cosmetic, observational, or unrelated to AE.
- There is no suggestion of margin involvement or need for further treatment.

**Key rule:**
If the addendum includes *any indication* that the current treatment is insufficient and further excision is under consideration or planned, classify as "Yes".

Output only: "Yes" or "No"
"""






 
    payload = {
        "messages": [
            {"role": "system", "content": system_context},
            {"role": "user", "content": f"Does this patient require additional excision? Clinical notes:\n{clinical_text}"},
        ],
        "temperature": 0.3,  # Lower temperature for more deterministic responses
        "top_p": 0.9,
        "max_tokens": 10
    }
    
    try:
        response = requests.post(ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"].strip()
    except Exception as e:
        print(f"Error calling GPT: {e}")
        return "Error"

In [325]:
# Use full Excel sheet
import pandas as pd
pd.set_option('display.max_colwidth', 50)

Cat = pd.read_excel(r'../De-Identified Mohs Full Data.xlsx',sheet_name='Identified Data',header=0) # Only Size at Greatest Dimension column
Cat.head()

,Randomized Study Number,Size at Greatest Dimension (S) ***Be sure this is from the note and not the Mohs episode!,Recurrent Tumor (R),"Location - Nose, Eyelid, Nail, Lip, Genitalia, or Acral (L)",Aggressive Tumor Pathology Type (AT),DFSP (D),Coordinated repair with ENT/oculoplastics (C),Additional C&F,Additional Excision,Multiple Sites,Business Days from Referral to First Appt,Physical Exam (PE),Assessment & Plan (A&P),Addendum (A)
0,36,0.8,Y,N,N,N,N,N,N,N,32.0,"Physical exam:\nGeneral: Well developed, well ...",1. Chronic actinic damage in the setting of hi...,Addendum 7/11/2023\n \nThe pathology results w...
1,240,0.6,Y,Y,N,N,N,N,N,N,40.0,Physical exam:\nGeneral: Well-dressed; well-no...,Procedure Orders\nBiopsy [486623795] ordered b...,Addendum 7/11/2023\nThe pathology results were...
2,32,0.3,N,N,N,N,N,N,N,N,4.0,Physical exam:\nGeneral: Well-dressed; well-no...,Assessment & Plan: \n1. Brown papule on the le...,Addendum 7/13/2023\n \nThe pathology results w...
3,111,6.5,N,N,N,N,Y,N,N,N,35.0,Physical exam:\nVital signs: There were no vit...,Assessment & Plan: \nBiopsy-proven basal cell ...,NaN
4,197,3.5,N,N,N,N,N,Y,Y,N,35.0,"Physical exam:\nGeneral: Well developed, well ...",Assessment & Plan: \n1. Chronic actinic sun d...,Addendum 7/5/2023\n \nThe pathology results we...


In [326]:
# Set options here
sample_size = len(Cat) # Change number to increase or decrease number of patients read from file
#sample_size = 10
multiple_columns = True # Set to True to use all input text columns, False to use only PE column

# Array of indexes containing patients with notes that have problems (Some may be ok, these are from size so a location could exist but not size)
skip_patients_array = [7,8,22,23,32,34,38,39,49,50,52,55,62,66,73,80,88,89,101,102,107,110,121,123,131,133,135,
                       136,137,140,150,155,157,164,170,184,187,193,200,206,210,214,227,228,229,234,252,261,265,279,287,292,293]
skip_patients = False # Set to True to skip bad patient notes, False to keep them in

In [327]:
# Takes the text in the PE, A&P, and A columns and asks ChatGPT to return "Yes" or "No" to the question
# May use only A column
import re

# Always split into 3 GPT calls, one for each column
def inputSplit(input_columns, que, index):
    while len(input_columns) < 3: # Some text columns may be empty so add a blank entry
        input_columns.append("")
        
    input_string = " ".join(map(str, input_columns)) # Apply str() to each index of input_columns then join with a space
    prompt = f"{que} \n\n {input_string}" # Create base prompt (question + input_columns)
    
    if(len(input_string) > 0):
        tmp_ans = [""] * len(input_columns)
        check_ans = [""] * len(input_columns)
        
        # Split into 3 GPT calls, one per column, and remove anything that isn't a "Yes" or "No" from the result
        for i in range(len(input_columns)):
            prompt = f"{que} \n\n {input_columns[i]}" # Recreate prompt with question + 1 column
            tmp_ans[i] = call_gpt(prompt) # Call GPT with prompt and save answer to array
            
            print(f"tmp_ans[{i}] = {str(tmp_ans[i])}")
            
            check_ans[i] = re.split(r'\s+|-', tmp_ans[i])  # Split by spaces or hyphens
            check_ans[i] = [re.sub(r'[^a-zA-Z]', '', j) for j in check_ans[i]]  # Remove non-alphabetic characters
            check_ans[i] = [j.lower() for j in check_ans[i]]  # Convert all to lowercase to handle variations

            check_ans[i] = [j for j in check_ans[i] if j in ['yes', 'no']] # Keep only "yes" or "no"
            
            print(f"check_ans[{i}] = {str(check_ans[i])}")
        if(check_ans[0] == check_ans[1] or check_ans[0] == check_ans[2]):
            if check_ans[0]:
                return tmp_ans[0]
            elif check_ans[1]:
                return tmp_ans[1]
            elif check_ans[2]:
                return tmp_ans[2]
        elif(check_ans[1] == check_ans[2]):
            if check_ans[1]:
                return tmp_ans[1]
            elif check_ans[2]:
                return tmp_ans[2]    


        # Count num of "yes" and "no"
        yes_count = [0] * len(input_columns)
        no_count = [0] * len(input_columns)
        
        for i in range(len(check_ans)):
            yes_count[i] = check_ans[i].count('yes')
            no_count[i] = check_ans[i].count('no')
            
        print(f"Yes counts: {yes_count}")
        print(f"No counts: {no_count}")
        
        # If no matches, return string that has the most "yes" or "no"
        if any(count > 0 for count in yes_count) or any(count > 0 for count in no_count): # Check count strings for any occurences or "yes" or "no"
            max_yes_no = [0] * len(input_columns)
            max_answer = 0
            max_i = -1
            
            # Check if string has more "yes" or "no"
            for i in range(len(input_columns)):
                max_yes_no[i] = max(yes_count[i], no_count[i])
                
            # Look for highest overall "yes" or "no" then return that string
            for i in range(len(input_columns)):
                if max_answer < max_yes_no[i]:
                    max_answer = max_yes_no[i]
                    max_i = i
            return tmp_ans[max_i]
        
        # All non "yes" or "no" answers, return empty string
        return ""
    else:
        # Somehow input_string is empty
        return "input_string empty"
    

true_results = [0 for i in range(sample_size)]
true_results_final = [0 for i in range(sample_size)]

output = [0 for i in range(sample_size)]
output_final = [0 for i in range(sample_size)]

for i in range(0, min(sample_size, len(Cat))):
    # If skip_patients is True and current index is in skip_patients_array, move to next index
    if skip_patients and i in skip_patients_array:
        continue
        
    true_results[i] = Cat.at[i, 'Additional Excision']
    que = que = "Does this case require an additional excision?"
    ans = ""
    
    if multiple_columns:
        input_columns = Cat.loc[i, ['Physical Exam (PE)', 'Assessment & Plan (A&P)', 'Addendum (A)']]
        ans = inputSplit(input_columns, que, i) # Result from GPT call
    elif not multiple_columns:
        input_columns = Cat.at[i, 'Addendum (A)']
        prompt = f"{que} \n\n {input_columns}"
        ans = call_gpt(prompt)
    
    print(f"Result[{i}] = {ans}")
    output[i] = ans

tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = Yes
check_ans[2] = ['yes']
Result[0] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = Yes
check_ans[1] = ['yes']
tmp_ans[2] = Yes
check_ans[2] = ['yes']
Result[1] = Yes
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = Yes
check_ans[2] = ['yes']
Result[2] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = Yes
check_ans[1] = ['yes']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[3] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = Yes
check_ans[2] = ['yes']
Result[4] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = Yes
check_ans[2] = ['yes']
Result[5] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = Yes
check_ans[2] = ['yes']
Result[6] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_

In [328]:
import re

# Go through every GPT response and turn it into "Y" or "N" to compare
for i in range(0, min(sample_size, len(Cat))):
    if skip_patients and i in skip_patients_array:
        continue
        
    yn = output[i]
    print(f"i={i} Answer: {yn}")
    yn_real = str(true_results[i])
    
    yn = re.split(r'\s+|-', yn)  # Split by spaces or hyphens
    yn = [re.sub(r'[^a-zA-Z]', '', j) for j in yn]  # Remove non-alphabetic characters
    yn = [j.lower() for j in yn]  # Convert all to lowercase to handle variations
    
    yn = [j for j in yn if j in ['yes', 'no']] # Keep only "yes" or "no"
    
    if len(yn) == 0:
        output_final[i] = "error_2"  # No valid response detected, bad prediction
    
    yes_count = yn.count('yes')
    no_count = yn.count('no')
    
    if yes_count > no_count:
        output_final[i] = "Y"
    elif no_count > yes_count:
        output_final[i] = "N"
        
    # Check if the given input is valid
    if yn_real.strip() and yn_real.lower() != 'nan':
        if 'y' in yn_real.lower():
            true_results_final[i] = "Y" # Some are input like N/Y or Y/N/N so look for Y and if exists compare with that
        else:
            true_results_final[i] = yn_real.strip()
    else:
        true_results_final[i] = "error_1" # No valid input provided in true_results, bad note
    
    print(f"Result[{i}]: ChatGPT Prediction = {output_final[i]} Answer Given = {true_results_final[i]}")

i=0 Answer: No
Result[0]: ChatGPT Prediction = N Answer Given = N
i=1 Answer: Yes
Result[1]: ChatGPT Prediction = Y Answer Given = N
i=2 Answer: No
Result[2]: ChatGPT Prediction = N Answer Given = N
i=3 Answer: No
Result[3]: ChatGPT Prediction = N Answer Given = N
i=4 Answer: No
Result[4]: ChatGPT Prediction = N Answer Given = Y
i=5 Answer: No
Result[5]: ChatGPT Prediction = N Answer Given = N
i=6 Answer: No
Result[6]: ChatGPT Prediction = N Answer Given = N
i=7 Answer: No
Result[7]: ChatGPT Prediction = N Answer Given = N
i=8 Answer: No
Result[8]: ChatGPT Prediction = N Answer Given = N
i=9 Answer: No
Result[9]: ChatGPT Prediction = N Answer Given = N
i=10 Answer: No
Result[10]: ChatGPT Prediction = N Answer Given = N
i=11 Answer: Yes
Result[11]: ChatGPT Prediction = Y Answer Given = N
i=12 Answer: No
Result[12]: ChatGPT Prediction = N Answer Given = N
i=13 Answer: No
Result[13]: ChatGPT Prediction = N Answer Given = N
i=14 Answer: No
Result[14]: ChatGPT Prediction = N Answer Given = 

In [329]:
# Configure pandas to show full text
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

for i in range(sample_size):
    if true_results[i] == "Y" and output[i] == "No":
        print(f"\n==== False Negative at index {i} ====")
        # Print full text of both columns
        print("[Addendum]:", Cat.at[i, 'Addendum (A)'])
        print("=" * 50)


==== False Negative at index 4 ====
[Addendum]: Addendum 7/5/2023
 
The pathology results were relayed to the patient's sister via phone call (see below). 
 
Lesion A: considered treated. No further intervention necessary.
 
Lesion B: Discussed therapeutic options, including Efudex and curettage and electrofulguration, and their associated risks and benefits. Sister opts for curettage and electrofulguration.
 
Lesion C:  Follow up for excision. Patient's sister denies the patient having artificial heart valves, artificial joints, or a cardiac stent within the last 3 months. Prophylactic antibiotics are not indicated.
 
Lesion D: Advised Mohs surgery due to size and location where tissue conservation is critical. Discussed risks (including but not limited to infection, bleeding, scar formation), benefits and alternatives as well as repair options that are often uncertain until the lesion has been excised, discussed that the surgical defect may be larger than the clinically evident lesi

In [330]:
# Configure pandas to show full text
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

for i in range(sample_size):
    if true_results[i] == "N" and output[i] == "Yes":
        print(f"\n==== False Positive at index {i} ====")
        # Print full text of both columns
        print("[Addendum]:", Cat.at[i, 'Addendum (A)'])
        print("=" * 50)


==== False Positive at index 1 ====
[Addendum]: Addendum 7/11/2023
The pathology results were relayed via phone call (see below). The patient is healing well without redness or discharge. 
 
Patient notified of biopsy result, advised Mohs surgery due to recurrent nature, ill defined margins, and location where tissue conservation is critical.  Discussed risks (including but not limited to infection, bleeding, scar formation), benefits and alternatives as well as repair options that are often uncertain until the lesion has been excised, discussed that the surgical defect may be larger than the clinically evident lesion.  Reviewed the Mohs surgery process and advised to reserve the entire day for the procedure.  Advised to continue all medications.  Preoperative antibiotics are  not indicated.  Phone Mohs completed and order placed for Mohs 3.0. 
 
Pathology: 
Results for orders placed or performed in visit on 07/05/23
DERMATOPATHOLOGY EXAM
Result	Value	Ref Range
 	Case Report	 	 
 	 	S

In [331]:
n_count = 0
for i in range(0, min((sample_size), len(Cat))):
    if output_final[i].strip() != true_results_final[i].strip() and true_results_final[i] != "error_1" and output[i] != "Yes":
        n_count += 1
        print(f"i = {i}: Unfiltered ChatGPT Prediction = {output[i]}\nUnfiltered AT Given = {true_results[i]}")
        print(f"i = {i}: ChatGPT Prediction = {output_final[i]} AT Given = {true_results_final[i]}")
        print()
        print("-"*40) # Spacer
print(f"n_count = {n_count}")

i = 4: Unfiltered ChatGPT Prediction = No
Unfiltered AT Given = Y
i = 4: ChatGPT Prediction = N AT Given = Y

----------------------------------------
i = 48: Unfiltered ChatGPT Prediction = No
Unfiltered AT Given = Y x 2
i = 48: ChatGPT Prediction = N AT Given = Y

----------------------------------------
i = 51: Unfiltered ChatGPT Prediction = No
Unfiltered AT Given = Y x 2
i = 51: ChatGPT Prediction = N AT Given = Y

----------------------------------------
i = 66: Unfiltered ChatGPT Prediction = No
Unfiltered AT Given = Y
i = 66: ChatGPT Prediction = N AT Given = Y

----------------------------------------
i = 78: Unfiltered ChatGPT Prediction = No
Unfiltered AT Given = Y x 2
i = 78: ChatGPT Prediction = N AT Given = Y

----------------------------------------
i = 94: Unfiltered ChatGPT Prediction = No
Unfiltered AT Given = Y
i = 94: ChatGPT Prediction = N AT Given = Y

----------------------------------------
i = 95: Unfiltered ChatGPT Prediction = No
Unfiltered AT Given = Y
i = 9

In [332]:
y_count = 0
for i in range(0, min((sample_size), len(Cat))):
    if output_final[i] != true_results_final[i] and true_results_final[i] != "error_1" and output[i] != "No":
        y_count += 1
        print(f"i = {i}: Unfiltered ChatGPT Prediction = {output[i]}\nUnfiltered AT Given = {true_results[i]}")
        print(f"i = {i}: ChatGPT Prediction = {output_final[i]} AT Given = {true_results_final[i]}")
        print()
        print("-"*40) # Spacer
print(f"y_count = {y_count}")

i = 1: Unfiltered ChatGPT Prediction = Yes
Unfiltered AT Given = N
i = 1: ChatGPT Prediction = Y AT Given = N

----------------------------------------
i = 11: Unfiltered ChatGPT Prediction = Yes
Unfiltered AT Given = N
i = 11: ChatGPT Prediction = Y AT Given = N

----------------------------------------
i = 32: Unfiltered ChatGPT Prediction = Yes
Unfiltered AT Given = N
i = 32: ChatGPT Prediction = Y AT Given = N

----------------------------------------
i = 34: Unfiltered ChatGPT Prediction = Yes
Unfiltered AT Given = N
i = 34: ChatGPT Prediction = Y AT Given = N

----------------------------------------
i = 207: Unfiltered ChatGPT Prediction = Yes
Unfiltered AT Given = N
i = 207: ChatGPT Prediction = Y AT Given = N

----------------------------------------
i = 279: Unfiltered ChatGPT Prediction = Yes
Unfiltered AT Given = N
i = 279: ChatGPT Prediction = Y AT Given = N

----------------------------------------
y_count = 6


In [333]:
count = 0
for i in range(0, min((sample_size), len(Cat))):
    if true_results_final[i] == "N":
        count += 1
print(f"no_count = {count}")
# 24 and 113 are bad notes for comparing, NA on every ground truth column

no_count = 269


In [334]:
def confusionMatrix():
    TP = TN = FP = FN = 0
    for i in range(0, min((sample_size), len(Cat))):
        if output_final[i] == true_results_final[i]: # True+/True-
            if output_final[i] == "Y": # True Positive
                TP += 1
            elif output_final[i] == "N": # True Negative
                TN += 1
        elif(output_final[i] != true_results_final[i] and true_results_final[i] != "error_1" and true_results_final[i] != "error_2"): # False+/False-
            if output_final[i] == "Y": # False Positive
                FP += 1
            elif output_final[i] == "N": # False Negative
                FN += 1
    return {"TP": TP, "TN": TN, "FP": FP, "FN": FN}

def confusionMetrics(confusion_matrix):
    TP = confusion_matrix["TP"]
    TN = confusion_matrix["TN"]
    FP = confusion_matrix["FP"]
    FN = confusion_matrix["FN"]
    
    total = TP + TN + FP + FN
    accuracy = (TP + TN) / (TP + TN + FP + FN) if total else 0 # How often did model predict correctly
    precision = TP / (TP + FP) if (TP + FP) else 0 # When model predicted Y, how often was it correct
    recall = TP / (TP + FN) if (TP + FN) else 0 # How many of the true Y cases did model correctly identify
    F1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0 # Balance between Precision and Recall
    
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "F1": F1}

            
total_size = 0
correct_predictions = 0
no_response_given = 0
no_alphabetical_result_returned = 0
bad_cases_combined = 0
nrg_not_narr = 0
narr_not_nrg = 0
# nrg = no valid response given in (L) column
# narr = no alphabetical result returned from ChatGPT output

for i in range(0, min((sample_size), len(Cat))):
    if(output_final[i] == "error_2" and true_results_final[i] == "error_1"):
        bad_cases_combined += 1
        
    if output_final[i] == "error_2":
        no_alphabetical_result_returned += 1
        if true_results_final[i] != "error_1":
            narr_not_nrg += 1
            
    if true_results_final[i] == "error_1":
        no_response_given += 1
        if output_final[i] != "error_2":
            nrg_not_narr += 1
    
    if output_final[i] == true_results_final[i]:
        correct_predictions += 1
        
    total_size += 1
    
accuracy = correct_predictions / total_size
print("# of Correct Predictions = " + str(correct_predictions))
print("Sample Size = " + str(total_size))
print("Accuracy = " + "{:.2f}%".format(accuracy * 100))
print()

# Accuracy when removing cases where the Location is not given (best to look at)
no_response_given_accuracy = correct_predictions / (total_size - no_response_given)
print("Number of Notes without a Given AE = " + str(no_response_given))
print("Accuracy = " +"{:.2f}%".format(no_response_given_accuracy * 100))
print()

# Accuracy when removing cases where GPT didn't return a valid alphabetical answer (not the best to look at, could be slightly lower)
no_alphabetical_result_returned_accuracy = correct_predictions / (total_size - no_alphabetical_result_returned)
print("Number of PE/A&P/A Notes without a valid GPT Given AE = " + str(no_alphabetical_result_returned))
print("Accuracy = " + "{:.2f}%".format(no_alphabetical_result_returned_accuracy * 100))
print()

# Accuracy when removing all above bad cases (ok to look at, could be slightly lower)
bad_cases_total_accuracy = correct_predictions / (total_size - (narr_not_nrg + nrg_not_narr + bad_cases_combined))
print("Number of Combined Notes without a Given AE = " + str(narr_not_nrg + nrg_not_narr + bad_cases_combined))
print("Accuracy = " + "{:.2f}%".format(bad_cases_total_accuracy * 100))
print()

confusion_matrix = confusionMatrix()
confusion_metrics = confusionMetrics(confusion_matrix)

print("Confusion Matrix:")
for name, value in confusion_matrix.items():
    print(f"{name}: {value}")

print("\nConfusion Matrix Metrics:")
for name, value in confusion_metrics.items():
    print(f"{name}: {(value * 100):.2f}")

# of Correct Predictions = 263
Sample Size = 294
Accuracy = 89.46%

Number of Notes without a Given AE = 3
Accuracy = 90.38%

Number of PE/A&P/A Notes without a valid GPT Given AE = 0
Accuracy = 89.46%

Number of Combined Notes without a Given AE = 3
Accuracy = 90.38%

Confusion Matrix:
TP: 0
TN: 263
FP: 6
FN: 22

Confusion Matrix Metrics:
accuracy: 90.38
precision: 0.00
recall: 0.00
F1: 0.00
